## Using linear regression for classification

* classifier를 구현하기 위한 가장 간단한 방법 중 하나는 linear regression algorithm을 수정하는 것이다.
* linear regression model은 f(x) = wx 같이 선형으로 보이는 함수들의 집합이며, 연속적인 실수를 입력으로 받아 연속적인 실수를 출력으로 생성한다.
* 기억할 것은 classification은 연속이 아닌 이산 출력이란 점이다.
* regression 모델이 두 가지 형태의 값을 출력할 경우, 어떠한 임계치를 주고 해당 임계치보다 높다면 1, 낮다면 0을 줄 수도 있다.

<img src="./fig/fig1.png" />
<img src="./fig/fig2.png" />

* 데이터를 line으로 모델링했기 때문에 입력은 0~1 사이의 값을 출력으로 나타낼 수 있다.
* 어떤 하나의 아이템이 하나의 카테고리에 속하는지 아니면 다른 카테고리에 속하는지를 결정해야 할 필요가 있으며, 대부분의 경우 0.5를 기준으로 판단한다.

In [ ]:
import tensorflow as tf
import numpy as np
import  matplotlib.pyplot as plt
%matplotlib inline

x_label0 = np.random.normal(7,1,10)
x_label1 = np.random.normal(2,1,10)
#x_label0 = np.append(np.random.normal(7,1,9),20)
xs = np.append(x_label0, x_label1)
labels = [0.] * len(x_label0) + [1.] * len(x_label1)

plt.scatter(xs, labels)

learning_rate = 0.001
training_epochs = 1000

X = tf.placeholder("float")
Y = tf.placeholder("float")

def model(X,w):
    return tf.add(tf.mul(w[1], tf.pow(X,1)),
                 tf.mul(w[0], tf.pow(X,0)))

w = tf.Variable([0., 0.], name="parameters")
y_model = model(X, w)
cost = tf.reduce_sum(tf.square(Y-y_model))
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for epoch in range(training_epochs):
    sess.run(train_op, feed_dict={X: xs, Y: labels})
    current_cost = sess.run(cost, feed_dict={X:xs, Y:labels})
    if epoch % 100 == 0:
        print(epoch, current_cost)
        
w_val = sess.run(w)
print('learning parameters', w_val)

correct_prediction = tf.equal(Y, tf.to_float(tf.greater(y_model, 0.5)))

test = tf.to_float(tf.greater(y_model, 0.5))

print(sess.run(test, feed_dict={X:xs}))
accuracy = tf.reduce_mean(tf.to_float(correct_prediction))

print('accuracy', sess.run(accuracy, feed_dict={X:xs, Y: labels}))

sess.close()

all_xs = np.linspace(0,10, 100)
plt.plot(all_xs, all_xs*w_val[1] + w_val[0])
plt.show()

## Using one-dimensional logistic regression

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

learning_rate = 0.01
training_epochs = 1000

def sigmoid(x):
    return 1. / (1. + np.exp(-x))

x1 = np.random.normal(-4, 2, 1000)
x2 = np.random.normal(4, 2, 1000)
xs = np.append(x1, x2)
ys = np.asarray([0.] * len(x1) + [1.] * len(x2))

plt.scatter(xs,ys)

X = tf.placeholder(tf.float32, shape=(None,), name="x")
Y = tf.placeholder(tf.float32, shape=(None,), name="y")
w = tf.Variable([0., 0.], name="parameter", trainable=True)
y_model = tf.sigmoid(-(w[1] * X + w[0]))
cost = tf.reduce_mean(-tf.log(y_model*Y + (1-y_model) * (1 - Y)))

train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    prev_err = 0
    for epoch in range(training_epochs):
        err, _ = sess.run([cost, train_op], {X:xs, Y:ys})
        print(epoch, err)
        if abs(prev_err - err) < 0.0001:
            break
        prev_err = err
    w_val = sess.run(w, {X:xs, Y: ys})
    
all_xs = np.linspace(-10, 10, 100)
plt.plot(all_xs, sigmoid(-(all_xs * w_val[1] + w_val[0])))
plt.show()

## Using two-dimensional logistic regression

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

learning_rate = 0.1
training_epochs = 2000

def sigmoid(x):
    return 1. / (1. + np.exp(-x))

x1_label1 = np.random.normal(3, 1, 1000)
x2_label1 = np.random.normal(2, 1, 1000)
x1_label2 = np.random.normal(7, 1, 1000)
x2_label2 = np.random.normal(6, 1, 1000)
x1s = np.append(x1_label1, x1_label2)
x2s = np.append(x2_label1, x2_label2)
ys = np.asarray([0.] * len(x1_label1) + [1.] * len(x1_label2))

X1 = tf.placeholder(tf.float32, shape=(None,), name="x1")
X2 = tf.placeholder(tf.float32, shape=(None,), name="x2")
Y = tf.placeholder(tf.float32, shape=(None,), name="y")
w = tf.Variable([0., 0., 0.], name="w", trainable=True)

y_model = tf.sigmoid(w[2] * X2 + w[1] * X1 + w[0])
cost = tf.reduce_mean(-tf.log(y_model * Y + (1 - y_model) * (1-Y)))
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    prev_err = 0
    for epoch in range(training_epochs):
        err, _ = sess.run([cost, train_op], {X1: x1s, X2:x2s, Y:ys})
        print(epoch, err)
        if abs(prev_err - err) < 0.0001:
            break
        prev_err = err
    w_val = sess.run(w, {X1: x1s, X2: x2s, Y: ys})
    
x1_boundary, x2_boundary = [], []
for x1_test in np.linspace(0, 10, 100):
    for x2_test in np.linspace(0, 10, 100):
        z = sigmoid(x2_test*w_val[2] + x1_test*w_val[1] + w_val[0])
        if abs(z - 0.5) < 0.01:
            x1_boundary.append(x1_test)
            x2_boundary.append(x2_test)
            
plt.scatter(x1_boundary, x2_boundary, c="b", marker="o", s=20)
plt.scatter(x1_label1, x2_label1, c="r", marker="x", s=20)
plt.scatter(x1_label2, x2_label2, c="g", marker="1", s=20)

plt.show()

## Visualizing multiclass data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x1_label0 = np.random.normal(1, 1, (100,1))
x2_label0 = np.random.normal(1, 1, (100,1))
x1_label1 = np.random.normal(5, 1, (100,1))
x2_label1 = np.random.normal(4, 1, (100,1))
x1_label2 = np.random.normal(8, 1, (100,1))
x2_label2 = np.random.normal(0, 1, (100,1))

plt.scatter(x1_label0, x2_label0, c='r', marker='o', s=60)
plt.scatter(x1_label1, x2_label1, c='g', marker='x', s=60)
plt.scatter(x1_label2, x2_label2, c='b', marker='_', s=60)
plt.show()

## Setting up training and test data for multiclass classification

In [ ]:
xs_label0 = np.hstack((x1_label0, x2_label0))
xs_label1 = np.hstack((x1_label1, x2_label1))
xs_label2 = np.hstack((x1_label2, x2_label2))
xs = np.vstack((xs_label0, xs_label1, xs_label2))

labels = np.matrix([[1., 0., 0.]] * len(x1_label0) + [[0., 1., 0.]] * len(x1_label1) + [[0., 0., 1.]] * len(x1_label2))

arr = np.arange(xs.shape[0])
np.random.shuffle(arr)

xs = xs[arr, :]

labels = labels[arr, :]

test_x1_label0 = np.random.normal(1, 1, (10,1))
test_x2_label0 = np.random.normal(1, 1, (10,1))
test_x1_label1 = np.random.normal(5, 1, (10,1))
test_x2_label1 = np.random.normal(4, 1, (10,1))
test_x1_label2 = np.random.normal(8, 1, (10,1))
test_x2_label2 = np.random.normal(0, 1, (10,1))
test_xs_label0 = np.hstack((test_x1_label0, test_x2_label0))
test_xs_label1 = np.hstack((test_x1_label1, test_x2_label1))
test_xs_label2 = np.hstack((test_x1_label2, test_x2_label2))

test_xs = np.vstack((test_xs_label0, test_xs_label1, test_xs_label2))
test_labels = np.matrix([[1., 0., 0.]] * 10 + [[0., 1., 0.]] * 10 + [[0., 0., 1.]] * 10)

train_size, num_features = xs.shape

## Using softmax regression

In [ ]:
import tensorflow as tf

learning_rate = 0.01
training_epochs = 1000
num_labels = 3
batch_size = 100

X = tf.placeholder("float", shape=[None, num_features])
Y = tf.placeholder("float", shape=[None, num_labels])

W = tf.Variable(tf.zeros([num_features, num_labels]))
b = tf.Variable(tf.zeros([num_labels]))
y_model = tf.nn.softmax(tf.matmul(X, W) + b)

cost = -tf.reduce_sum(Y * tf.log(y_model))
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(y_model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

## Executing the graph

In [ ]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for step in range(training_epochs * train_size // batch_size):
        offset = (step * batch_size) % train_size
        batch_xs = xs[offset:(offset+batch_size), :]
        batch_labels = labels[offset:(offset+batch_size)]
        err,_ = sess.run([cost, train_op], feed_dict={X: batch_xs, Y: batch_labels})
        print (step,err)
        W_val = sess.run(W)
   #     print ('w', W_val)
        b_val = sess.run(b)
   #     print ('b', b_val)
   #     print "accuracy", accuracy.eval(feed_dict={X: test_xs, Y: test_labels})

In [ ]:
10 / 2